In [1]:
using CSV, DataFrames, StatsBase, BioSequences, FASTX

In [2]:
struct Feature
    type::String
    start::Int
    stop::Int
    strand::Char
    phase::Int
    attributes::Dict{String, String}
end

struct Gene
    id::String
    gene::Feature
    mRNAs::Vector{Feature}
    tRNAs::Vector{Feature}
    rRNAs::Vector{Feature}
    ncRNAs::Vector{Feature}
    CDSs::Vector{Feature}
    exons::Vector{Feature}
    introns::Vector{Feature}
    esites::Vector{Feature}
end

function attributes2dict(attribute_string::String)
    dict = Dict{String, String}()
    attributes = split(attribute_string, ";")
    for attribute in attributes
        kvs = split(attribute, "=")
        dict[first(kvs)] = last(kvs)
    end
    dict
end

function createFeature(row)
    Feature(row.feature, row.start, row.stop, row.strand[1], row.phase == "." ? 0 : parse(Int, row.phase), attributes2dict(row.attributes))
end


createFeature (generic function with 1 method)

In [3]:
function add_feature(gene, feature)
    #check feature is on same strand as the gene
    @assert feature.strand == gene.gene.strand "strand error: $(string(feature)) $(string(gene.gene))"
    #check feature is within gene
    @assert feature.start >= gene.gene.start && feature.stop <= gene.gene.stop "boundary error: $(string(feature)) $(string(gene.gene))"
    if feature.type == "mRNA"
        push!(gene.mRNAs, feature)
    elseif feature.type == "tRNA"
        push!(gene.tRNAs, feature)
    elseif feature.type == "rRNA"
        push!(gene.rRNAs, feature)
    elseif feature.type == "ncRNA"
        push!(gene.ncRNAs, feature)
    elseif feature.type == "CDS"
        push!(gene.CDSs, feature)
    elseif feature.type == "exon"
        push!(gene.exons, feature)
    elseif feature.type == "intron"
        push!(gene.introns, feature)
    elseif feature.type == "misc_feature"
        push!(gene.esites, feature)
    else
        println("unknown feature type: ", feature.type)
    end
end

add_feature (generic function with 1 method)

In [4]:
function add_features(feature_type::String)
    gff_features = filter(x->x.feature == feature_type, gff)
    for row in eachrow(gff_features)
        f = createFeature(row)
        gene = get(f.attributes, "gene", nothing)
        if isnothing(gene)
            println("no gene attribute for ", f.attributes["ID"])
        else
            mygene = findfirst(x -> x.id == gene, genes) 
            if isnothing(mygene)
                println("no gene found matching ", gene)
            else
                add_feature(genes[mygene], f)
            end
        end
    end
end

add_features (generic function with 1 method)

In [5]:
ref = FASTA.Reader(open("Phylloglossum_drummondii_chloroplast.fasta")) do infile; first(infile); end
refseq = FASTA.sequence(LongDNA{4}, ref)
refseqcomp = complement(refseq)

gff = CSV.File("Phylloglossum_drummondii_chloroplast.gff", comment = "#", header = ["sequence", "software", "feature", "start", "stop", "score", "strand", "phase", "attributes"]) |> DataFrame

Row,sequence,software,feature,start,stop,score,strand,phase,attributes
,String,String15,String15,Int64,Int64,String15,String1,String1,String
1,Phylloglossum_drummondii_chloroplast,Geneious,region,1,144520,.,+,0,Is_circular=true
2,Phylloglossum_drummondii_chloroplast,Chloe,intron,290,749,.,+,.,gene=rps12B;ID=rps12B/1/intron/1
3,Phylloglossum_drummondii_chloroplast,Chloe,gene,290,1007,.,+,.,Name=rps12B;ID=rps12B;NCBI Feature Key=gene
4,Phylloglossum_drummondii_chloroplast,Pyrimid,misc_feature,737,737,1.02E-295,.,.,note=C to U RNA editing (frequency 0.87)
5,Phylloglossum_drummondii_chloroplast,Chloe,CDS,750,1007,.,+,0,codon_start=1;gene=rps12B;ID=rps12B/1/CDS/2
6,Phylloglossum_drummondii_chloroplast,Chloe,mRNA,1060,1512,.,+,.,gene=rps7;ID=rps7.mRNA
7,Phylloglossum_drummondii_chloroplast,Chloe,CDS,1060,1512,.,+,0,codon_start=1;gene=rps7;ID=rps7/1/CDS/1
8,Phylloglossum_drummondii_chloroplast,Chloe,gene,1060,1512,.,+,.,Name=rps7;ID=rps7;NCBI Feature Key=gene
9,Phylloglossum_drummondii_chloroplast,Chloe,mRNA,1784,3947,.,+,.,gene=ndhB;ID=ndhB.mRNA


In [6]:
gffgenes = filter(x->x.feature == "gene", gff)
genes = Gene[]
for row in eachrow(gffgenes)
    gene = createFeature(row)
    id = get(gene.attributes, "ID", nothing)
    if isnothing(id)
        println("no ID for ", gene.attributes["Name"])
    else
        push!(genes, Gene(gene.attributes["ID"], gene, Feature[], Feature[], Feature[], Feature[], Feature[], Feature[], Feature[], Feature[]))
    end
end
genes

132-element Vector{Gene}:
 Gene("rps12B", Feature("gene", 290, 1007, '+', 0, Dict("NCBI Feature Key" => "gene", "Name" => "rps12B", "ID" => "rps12B")), Feature[], Feature[], Feature[], Feature[], Feature[], Feature[], Feature[], Feature[])
 Gene("rps7", Feature("gene", 1060, 1512, '+', 0, Dict("NCBI Feature Key" => "gene", "Name" => "rps7", "ID" => "rps7")), Feature[], Feature[], Feature[], Feature[], Feature[], Feature[], Feature[], Feature[])
 Gene("ndhB", Feature("gene", 1784, 3947, '+', 0, Dict("NCBI Feature Key" => "gene", "Name" => "ndhB", "ID" => "ndhB")), Feature[], Feature[], Feature[], Feature[], Feature[], Feature[], Feature[], Feature[])
 Gene("trnL-CAA", Feature("gene", 4759, 4839, '+', 0, Dict("NCBI Feature Key" => "gene", "Name" => "trnL-CAA", "ID" => "trnL-CAA")), Feature[], Feature[], Feature[], Feature[], Feature[], Feature[], Feature[], Feature[])
 Gene("psbM", Feature("gene", 5220, 5324, '+', 0, Dict("NCBI Feature Key" => "gene", "Name" => "psbM", "ID" => "psbM")), 

In [7]:
#check for duplicate genes
geneids = [x.id for x in genes]
if length(geneids) ≠ length(unique(geneids))
    println("gene IDs contain duplicates")
    println(filter(x -> last(x) > 1, countmap(geneids)))
end

In [8]:
add_features("tRNA")
add_features("rRNA")
add_features("ncRNA")
add_features("CDS")
for gene in genes
    if gene.gene.strand == '-'
        reverse!(gene.CDSs)
    end
end

In [9]:
genes

132-element Vector{Gene}:
 Gene("rps12B", Feature("gene", 290, 1007, '+', 0, Dict("NCBI Feature Key" => "gene", "Name" => "rps12B", "ID" => "rps12B")), Feature[], Feature[], Feature[], Feature[], Feature[Feature("CDS", 750, 1007, '+', 0, Dict("ID" => "rps12B/1/CDS/2", "gene" => "rps12B", "codon_start" => "1"))], Feature[], Feature[], Feature[])
 Gene("rps7", Feature("gene", 1060, 1512, '+', 0, Dict("NCBI Feature Key" => "gene", "Name" => "rps7", "ID" => "rps7")), Feature[], Feature[], Feature[], Feature[], Feature[Feature("CDS", 1060, 1512, '+', 0, Dict("ID" => "rps7/1/CDS/1", "gene" => "rps7", "codon_start" => "1"))], Feature[], Feature[], Feature[])
 Gene("ndhB", Feature("gene", 1784, 3947, '+', 0, Dict("NCBI Feature Key" => "gene", "Name" => "ndhB", "ID" => "ndhB")), Feature[], Feature[], Feature[], Feature[], Feature[Feature("CDS", 1784, 2509, '+', 0, Dict("ID" => "ndhB/1/CDS/1", "gene" => "ndhB", "codon_start" => "1")), Feature("CDS", 3189, 3947, '+', 0, Dict("ID" => "ndhB/1/CDS/3

In [10]:
function calculate_codon_position(gene::Gene, position::Int)
    @assert position >= gene.gene.start && position <= gene.gene.stop
    sort!(gene.CDSs, by=x->x.start)
    if gene.gene.strand == '-'; reverse!(gene.CDSs); end
    seq = LongDNA{4}()
    cds_position = 0
    in_cds = false
    for cds in gene.CDSs
        if gene.gene.strand == '+' && position >= cds.stop
            cds_position += cds.stop - cds.start + 1
        elseif gene.gene.strand == '-' && position <= cds.start
            cds_position += cds.stop - cds.start + 1
        elseif gene.gene.strand == '+' && position >= cds.start
            cds_position += position - cds.start + 1
            in_cds = true
        elseif gene.gene.strand == '-' && position <= cds.stop
            cds_position += cds.stop - position + 1
            in_cds = true
        end
        append!(seq, gene.gene.strand == '+' ? LongDNA{4}(refseq[cds.start:cds.stop]) : reverse_complement(LongDNA{4}(refseq[cds.start:cds.stop])))
    end
    codon_no = codon_position = 0
    codon = LongDNA{4}([DNA_Gap, DNA_Gap, DNA_Gap])
    if in_cds
        codon_no = ceil(Int, cds_position/3.0)
        codon_position = cds_position - 3 * (codon_no - 1)
        codon = seq[3 * codon_no - 2:3 * codon_no]
    else
        cds_position = gene.gene.strand == '+' ? position - gene.gene.start + 1 : gene.gene.stop - position + 1
    end
    return cds_position, codon_no, codon_position, codon
end

calculate_codon_position (generic function with 1 method)

In [11]:
dna_stops = LongDNA{4}.([[DNA_T, DNA_A, DNA_A], [DNA_T, DNA_G, DNA_A], [DNA_T, DNA_A, DNA_G]])
rna_stops = convert.(LongRNA{4}, dna_stops)

3-element Vector{LongSequence{RNAAlphabet{4}}}:
 UAA
 UGA
 UAG

In [12]:
function prepact_name(position, gene, base, cds_position, aa, edited_aa)
    if ismissing(gene) || haskey(gene.gene.attributes, "pseudo") || haskey(gene.gene.attributes, "pseudogene")
        return string(position) * "e" * string(base)
    elseif aa == AA_Gap
        return gene.id * "e" * string(base) * string(cds_position)
    else
        return gene.id * "e" * string(base) * string(cds_position) * string(aa) * string(edited_aa)
    end
end

prepact_name (generic function with 1 method)

In [13]:
esites = DataFrame(genome = String[], strand = Char[], position = Int[], id = String[], reference_base = DNA[], edited_base = RNA[], proportion_edited = Float64[], gene = String[], cds_position = Int[], codon_position = Int[],
    codon = LongDNA{4}[], edited_codon = LongRNA{4}[], aa = AminoAcid[], edited_aa = AminoAcid[], synonymous = Union{Missing, Bool}[], creates_start = Bool[], creates_stop = Bool[], removes_stop = Bool[],
     preceding_base = RNA[], subsequent_base = RNA[])
gff_esites = filter(x->occursin("RNA editing", x.attributes), gff)
for row in eachrow(gff_esites)
    genome = "cp"
    position = row.start
    refbase = refseq[position]
    strand = '-'
    reference_base = complement(refbase)
    if refbase == DNA_C || refbase == DNA_T
        strand = '+'
        reference_base = refbase
    end
    edited_base = RNA(row.attributes[11])
    if refbase == DNA_C
        @assert edited_base == RNA_U
    elseif refbase == DNA_T
        @assert edited_base == RNA_C
    end
    proportion_edited = parse(Float64, row.attributes[end-5:end-2])
    mygene = missing
    gene = ""
    cds_position = codon_number = codon_position = 0
    blank = LongDNA{4}([DNA_Gap, DNA_Gap, DNA_Gap])
    codon = blank
    for g in reverse(genes)
        if position >= g.gene.start && position <= g.gene.stop && strand == g.gene.strand
            mygene = g
            gene = mygene.id
            cds_position, codon_number, codon_position, codon = calculate_codon_position(mygene, position)
            break
        end
    end
    edited_codon = LongRNA{4}([RNA_Gap, RNA_Gap, RNA_Gap])
    aa = AA_Gap
    edited_aa = AA_Gap
    synonymous = missing
    creates_start = creates_stop = removes_stop = false
    if codon ≠ blank
        edited_codon = convert(LongRNA{4}, codon)
        edited_codon[codon_position] = edited_base
        aa = BioSequences.translate(codon)[1]
        edited_aa = BioSequences.translate(edited_codon)[1]
        synonymous = aa == edited_aa
        creates_start = codon_number == 1 && edited_codon == LongRNA{4}([RNA_A, RNA_U, RNA_G])
        creates_stop = position == (strand == '+' ? mygene.gene.stop - 2 : mygene.gene.start + 2)
        removes_stop = codon ∈ dna_stops
    end
    preceding_base = strand == '+' ? refseq[position - 1] : refseqcomp[position + 1]
    subsequent_base = strand == '+' ? refseq[position + 1] : refseqcomp[position - 1]
    id = prepact_name(position, mygene, edited_base, cds_position, aa, edited_aa)
    push!(esites, (genome, strand, position, id, reference_base, edited_base, proportion_edited, gene, cds_position, codon_position, codon, edited_codon, aa, edited_aa,
     synonymous, creates_start, creates_stop, removes_stop, preceding_base, subsequent_base))
end
esites


Row,genome,strand,position,id,reference_base,edited_base,proportion_edited,gene,cds_position,codon_position,codon,edited_codon,aa,edited_aa,synonymous,creates_start,creates_stop,removes_stop,preceding_base,subsequent_base
,String,Char,Int64,String,DNA,RNA,Float64,String,Int64,Int64,LongSequ…,LongSequ…,AminoAcid,AminoAcid,Bool?,Bool,Bool,Bool,RNA,RNA
1,cp,+,737,rps12BeU448,C,U,0.8,rps12B,448,0,---,---,-,-,missing,false,false,false,U,G
2,cp,+,2178,ndhBeU395SL,C,U,0.9,ndhB,395,2,TCA,UUA,S,L,false,false,false,false,U,A
3,cp,+,2430,ndhBeU647PL,C,U,0.9,ndhB,647,2,CCT,CUU,P,L,false,false,false,false,C,U
4,cp,+,3241,ndhBeU779SL,C,U,0.8,ndhB,779,2,TCA,UUA,S,L,false,false,false,false,U,A
5,cp,+,3260,ndhBeU798II,C,U,0.7,ndhB,798,3,ATC,AUU,I,I,true,false,false,false,U,U
6,cp,+,3262,ndhBeU800SL,C,U,0.7,ndhB,800,2,TCA,UUA,S,L,false,false,false,false,U,A
7,cp,+,3574,ndhBeU1112PL,C,U,0.9,ndhB,1112,2,CCT,CUU,P,L,false,false,false,false,C,U
8,cp,+,3607,ndhBeU1145PL,C,U,0.9,ndhB,1145,2,CCA,CUA,P,L,false,false,false,false,C,A
9,cp,+,3974,3974eU,C,U,0.2,,0,0,---,---,-,-,missing,false,false,false,U,U


In [14]:
CSV.write("cp editing events.tsv", esites, delim='\t')

"cp editing events.tsv"

In [15]:
nrow(esites)

285

In [16]:
countmap(esites.codon_position)

Dict{Int64, Int64} with 4 entries:
  0 => 50
  2 => 188
  3 => 23
  1 => 24

In [18]:
count(skipmissing(esites.synonymous) .== true)

23

In [19]:
count(esites.creates_start)

4

In [20]:
count(esites.creates_stop)

1

In [19]:
countmap(esites.preceding_base)

Dict{RNA, Int64} with 4 entries:
  RNA_C => 84
  RNA_U => 159
  RNA_A => 38
  RNA_G => 4

In [22]:
mean(esites.proportion_edited)

0.7554385964912277